# Pointers

In [1]:
#include <iostream>
using std::cout, std::endl;

In [2]:
class HasInt {
    int the_int;
    public:
    HasInt(int value) : the_int(value) {}
    void set_int(int new_value) { the_int = new_value; }
    int get_int() const { return the_int; }
}

In [3]:
HasInt first(8);
first.get_int()

8

In [4]:
HasInt second = first;
second.get_int()

8

In [5]:
second.set_int(3);
second.get_int()

3

In [6]:
first.get_int()

8

In [2]:
template <class T>
void where(T const& thing) {
    printf("%p\n", &thing);
}

In [46]:
where(first)

0x400917b02c


In [47]:
where(second)

0x400917b030


## C++ Memory Model

- When you declare a variable, you are declaring a chunk of RAM big enough for the variable type
- When you assign a variable a new value, you change the bytes at that variable's location in RAM

```c++
HasInt second = first;
```
- Declares a new chunk of RAM big enough for an `HasInt`
- **Copies** the bytes from the RAM for `first` to the RAM for `second`

Python and Java are different. The same syntax as above:

```python
second = first
```
or 
```java
HasInt second = first;
``` 
make `first` and `second` refer to the same object; changes on that one object are reflected through either variable name.

In C++, you achieve similar behavior to Python or Java using **pointers**.

## Pointers

In [10]:
HasInt* one = new HasInt(25);
one->get_int()

25

- `HasInt*` is the type for "a pointer to a HasInt"
- `new` creates space for the `HasInt` on the heap, initializes the object, and returns a **pointer** to that space in memory
- `->` is how you access information through a pointer to an object

In [11]:
HasInt* two = one;
two->get_int()

25

In [12]:
two->set_int(42);
two->get_int()

42

In [13]:
one->get_int()

42

In [14]:
(*one).get_int()

42

- you can also use the unary `*` operator to get the object a pointer points to

So, if `one` and `two` point to the same object (i.e. the same chunk of RAM), then...

In [15]:
where(*one)

0x4027e6e810


In [16]:
where(*two)

0x4027e6e810


In [3]:
/*
  This method prints the bytes in RAM that a given variable has
*/
template <class T>
void bytes(T const& thing) {
    unsigned char* addr = (unsigned char*)&thing;
    printf("0x");
    for (int i = sizeof(T) - 1; i >= 0; i--) {
        printf("%02x", addr[i]);
    }
    printf("\n");
}

In [49]:
bytes(first)

0x00000008


In [50]:
bytes(second)

0x00000003


In [51]:
bytes(*one)

0x0000002a


In [52]:
bytes(*two)

0x0000002a


0x2A (hexidecimal) is the same as $2 * 16 + 10 = 42$

In [53]:
bytes(one)

0x0000004027e6e810


In [54]:
where(*one)

0x4027e6e810


- Draw this out!
- `one` exists somewhere. It's a `HasInt*`. It has information, encoded in bytes.
- `one` is a pointer to a `HasInt`. That `HasInt` exists somewhere and has data. That somewhere has an address.
- They bytes that `one` stores are the address to the `HasInt`!

In [55]:
bytes(two)

0x0000004027e6e810


In [56]:
where(*two)

0x4027e6e810


All data are stored as bytes in RAM.

Each byte has an address (or how would you find it?)

That address is data, and all data are stored as bytes, so an address is also bytes.

Variables are names for places in RAM. There is an address for every variable. 

Because addresses are data, you can store them in a variable.

A variable that stores an address is called a **pointer**!

In [58]:
int* number = new int(5);
where(number);
bytes(number);
where(*number);
bytes(*number);

0x400917b048
0x0000004029c88ed0
0x4029c88ed0
0x00000005


In [61]:
HasInt* a = new HasInt(2);
HasInt* b = new HasInt(1);
HasInt* c = a;
bytes(a);
bytes(b);
bytes(c);

0x000000402983ea00
0x000000402841e7c0
0x000000402983ea00


In [62]:
c = b;
bytes(a);
bytes(b);
bytes(c);

0x000000402983ea00
0x000000402841e7c0
0x000000402841e7c0


- remember, what does `=` do in c++?
  - it **copies** the bytes from one variable to another
- so when you assign one pointer to another, you copy the address (as bytes) of other to the first

In C++, there are bytes stored in RAM. That's it.

Every variable stores its own bytes.

If you want to have two variables refer to the same bytes, you need pointers, i.e.:

- Have two different variables that both store the address to the same object.

Variables that store addresses are called **pointers**

## `nullptr`

In [65]:
HasInt* foo = nullptr;  // Similar to None in Python or null in Java

In [63]:
bytes(nullptr)

0x0000000000000000


## `singly_linked.cpp`

- `struct`
- nested class/struct `Node`
  - existence of class if private to `SLList`
  - is in scope of the outer `template`
  - shouldn't expose `Node` in any of the public methods
- pointer references `Node *&node`
  - allows changes to the pointer
  - you can pass in a pointer with value `nullptr` and make the value not be `nullptr` anymore
  - Java and Python do not have an equivalent concept of passing by reference
- deliberate choice of recursive implementation
  - familiar to CS 111 students
  - similar to encouraged BST/AVL implementation
  - start the conversation on `Node *&node`
  - understand both `_add` and `_clear`
    - note the post-recursion logic in `_clear`
- Destructor `~SSList`
  - to clean up memory allocated by the class with `new`
  - `delete`